# Imports

In [122]:
# Bibliotecas padrão
import numpy as np
import pandas as pd

# PySpark - Sessão
from pyspark.sql import SparkSession

# PySpark - Funções
from pyspark.sql.functions import (
    concat_ws,
    avg,
    col,
    count,
    lit,
    log,
    percentile_approx,
    sum as Fsum,
    udf,
    when,
    to_date,
    lpad,
)

# PySpark - Tipos
from pyspark.sql.types import IntegerType, StringType

# PySpark - Janela
from pyspark.sql.window import Window


✅ 2. Separar as proporções em dev (ref) e oot (compare)

Agora vamos pivotar para ter proportion_dev e proportion_oot em colunas separadas:

✅ 2. Separar as proporções em dev (ref) e oot (compare)

Agora vamos pivotar para ter proportion_dev e proportion_oot em colunas separadas:

# Simulando nosso dataset

Ótimo! Vamos construir juntos uma simulação com os seguintes critérios:

🎯 Objetivo:
	•	Simular dois modelos (modelo_a, modelo_b)
	•	Cada um com dados nos ambientes:
	•	DEV: 6 meses
	•	OOT: 3 meses
	•	PRD: 12 meses
	•	modelo_a permanece calibrado
	•	modelo_b vai descalibrando ao longo do tempo

⸻

📦 Etapas da simulação:
	1.	Gerar uma base com colunas: score, env, year, month, model
	2.	modelo_a: score estável em todos os ambientes
	3.	modelo_b: score muda ao longo do tempo (ex: média ou variância cresce em PRD)

⸻

🐍 Código de simulação (usando Pandas)

In [23]:
import pandas as pd
import numpy as np

def simulate_model_data(model_name, start_year=2023):
    """
    Simula scores e variável resposta para um modelo nos ambientes DEV, OOT e PRD.
    
    modelo_a: score calibrado e estável
    modelo_b: score descalibrando em PRD (aumenta média ao longo do tempo)
    """
    rows = []
    rng = np.random.default_rng(seed=42 if model_name == 'modelo_a' else 99)

    def generate_block(env, year, month, loc, size=1000):
        # Gera scores com média loc e desvio padrão 0.1
        scores = rng.normal(loc=loc, scale=0.1, size=size)
        scores = np.clip(scores, 0.01, 0.99)  # garantir intervalo válido [0, 1]
        vr = rng.binomial(1, p=scores)  # variável resposta: simula desfecho com base no score
        return pd.DataFrame({
            'model': model_name,
            'env': env,
            'year': year,
            'month': month,
            'score': scores,
            'vr': vr
        })

    # Ambiente DEV (6 meses estáveis)
    for month in range(1, 7):
        rows.append(generate_block('DEV', start_year, month, loc=0.5))

    # Ambiente OOT (3 meses)
    for month in range(7, 10):
        loc = 0.5 if model_name == 'modelo_a' else 0.55  # modelo_b levemente deslocado
        rows.append(generate_block('OOT', start_year, month, loc=loc))

    # Ambiente PRD (12 meses)
    for month in range(1, 13):
        year = start_year + 1
        loc = 0.5 if model_name == 'modelo_a' else 0.5 + 0.02 * month  # modelo_b descalibra gradualmente
        rows.append(generate_block('PRD', year, month, loc=loc))

    return pd.concat(rows, ignore_index=True)

# Gerar os dados simulados
df_a = simulate_model_data('modelo_a')
df_b = simulate_model_data('modelo_b')

# Unir os dois modelos em um único DataFrame
df = pd.concat([df_a, df_b], ignore_index=True)

# Visualizar primeiros registros
print(df.head())


      model  env  year  month     score  vr
0  modelo_a  DEV  2023      1  0.530472   0
1  modelo_a  DEV  2023      1  0.396002   0
2  modelo_a  DEV  2023      1  0.575045   1
3  modelo_a  DEV  2023      1  0.594056   1
4  modelo_a  DEV  2023      1  0.304896   0


🧪 Colunas do DataFrame final:

* model: "modelo_a" ou "modelo_b"
* env: ambiente (DEV, OOT, PRD)
* year: ano
* month: mês
* score: probabilidade estimada pelo modelo (entre 0 e 1)
* vr: variável resposta (0 ou 1), simulada com base no score

In [24]:
df

,model,env,year,month,score,vr
0,modelo_a,DEV,2023,1,0.530472,0
1,modelo_a,DEV,2023,1,0.396002,0
2,modelo_a,DEV,2023,1,0.575045,1
3,modelo_a,DEV,2023,1,0.594056,1
4,modelo_a,DEV,2023,1,0.304896,0
...,...,...,...,...,...,...
41995,modelo_b,PRD,2024,12,0.518542,1
41996,modelo_b,PRD,2024,12,0.733841,0
41997,modelo_b,PRD,2024,12,0.799363,1
41998,modelo_b,PRD,2024,12,0.708009,0




✅ Resultado

Temos um DataFrame com:
	•	2 modelos (modelo_a e modelo_b)
	•	Score gerado mês a mês em diferentes ambientes
	•	modelo_b vai ficando cada vez mais diferente de DEV no ambiente de produção

⸻

In [50]:
# Initialize Spark Session
spark = SparkSession.builder.appName("GroupByCustomFunction").getOrCreate()

spkdf = spark.createDataFrame(df)
spkdf.show(10)

+--------+---+----+-----+-------------------+---+
|   model|env|year|month|              score| vr|
+--------+---+----+-----+-------------------+---+
|modelo_a|DEV|2023|    1| 0.5304717079754432|  0|
|modelo_a|DEV|2023|    1| 0.3960015893759504|  0|
|modelo_a|DEV|2023|    1| 0.5750451195806457|  1|
|modelo_a|DEV|2023|    1| 0.5940564716391213|  1|
|modelo_a|DEV|2023|    1|0.30489648113461637|  0|
|modelo_a|DEV|2023|    1| 0.3697820493137682|  0|
|modelo_a|DEV|2023|    1| 0.5127840403167285|  0|
|modelo_a|DEV|2023|    1|0.46837574076564176|  1|
|modelo_a|DEV|2023|    1|0.49831988424957113|  1|
|modelo_a|DEV|2023|    1|  0.414695607242642|  0|
+--------+---+----+-----+-------------------+---+
only showing top 10 rows

